# Med-I-C: AMR-Guard - Infection Lifecycle Orchestrator

## MedGemma Impact Challenge Submission

This notebook demonstrates the **Med-I-C** multi-agent system for antimicrobial stewardship:

**4-Agent Architecture:**
1. **Intake Historian** - Parse patient data, calculate CrCl, identify risk factors
2. **Vision Specialist** - Extract structured data from lab reports (any language)
3. **Trend Analyst** - Detect MIC creep and resistance velocity
4. **Clinical Pharmacologist** - Final Rx recommendations with safety checks

**Two Pathways:**
- **Stage 1 (Empirical)**: Agent 1 → Agent 4 (before lab results)
- **Stage 2 (Targeted)**: Agent 1 → Agent 2 → Agent 3 → Agent 4 (with lab results)

---

## 1. Environment Setup

In [ ]:
# Check GPU availability
import subprocess
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout)

In [ ]:
%%capture
# Install dependencies
!pip install -q langgraph>=0.0.15 langchain>=0.3.0 langchain-text-splitters
!pip install -q chromadb>=0.4.0 sentence-transformers
!pip install -q transformers>=4.50.0 torch accelerate bitsandbytes
!pip install -q pydantic>=2.0 python-dotenv openpyxl requests pypdf pandas
!pip install -q huggingface_hub

In [ ]:
import os
import sys
import json
import logging
from pathlib import Path
from typing import Any, Dict, List, Optional, Literal

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger('MedIC')

## 2. Hugging Face Authentication

MedGemma and TxGemma require accepting the license on Hugging Face.

1. Go to https://huggingface.co/google/medgemma-4b-it and accept the license
2. Go to https://huggingface.co/google/txgemma-2b-predict and accept the license
3. Add your HF token to Kaggle Secrets as `HF_TOKEN`

In [ ]:
# Authenticate with Hugging Face
from huggingface_hub import login

# Try to get token from Kaggle secrets
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
    print("Using HF token from Kaggle secrets")
except:
    # Fallback: set your token here for local testing
    HF_TOKEN = os.environ.get("HF_TOKEN", "")
    if HF_TOKEN:
        print("Using HF token from environment")
    else:
        print("WARNING: No HF token found. You may need to authenticate manually.")

if HF_TOKEN:
    login(token=HF_TOKEN)

## 3. Model Configuration & Loading

In [ ]:
# Model configuration
MODEL_CONFIG = {
    "medgemma_4b": {
        "model_id": "google/medgemma-4b-it",
        "description": "MedGemma 4B Instruction-Tuned - Primary model for all agents",
        "use_4bit": True,  # Use 4-bit quantization for memory efficiency
    },
    "medgemma_27b": {
        "model_id": "google/medgemma-27b-text-it",
        "description": "MedGemma 27B Text IT - For complex trend analysis (requires high VRAM)",
        "use_4bit": True,
    },
    "txgemma_9b": {
        "model_id": "google/txgemma-9b-predict",
        "description": "TxGemma 9B - Drug safety validation",
        "use_4bit": True,
    },
    "txgemma_2b": {
        "model_id": "google/txgemma-2b-predict",
        "description": "TxGemma 2B - Lightweight safety checker fallback",
        "use_4bit": False,  # Small enough to run without quantization
    },
}

# Display model info
print("Available Models:")
for name, config in MODEL_CONFIG.items():
    print(f"  - {name}: {config['description']}")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Cache for loaded models
_model_cache = {}
_tokenizer_cache = {}

def load_model(model_name: str = "medgemma_4b", force_reload: bool = False):
    """
    Load a model from Hugging Face with optional 4-bit quantization.
    
    Args:
        model_name: Key from MODEL_CONFIG
        force_reload: Force reload even if cached
    
    Returns:
        Tuple of (model, tokenizer)
    """
    global _model_cache, _tokenizer_cache
    
    if not force_reload and model_name in _model_cache:
        print(f"Using cached {model_name}")
        return _model_cache[model_name], _tokenizer_cache[model_name]
    
    config = MODEL_CONFIG.get(model_name)
    if not config:
        raise ValueError(f"Unknown model: {model_name}. Available: {list(MODEL_CONFIG.keys())}")
    
    model_id = config["model_id"]
    use_4bit = config.get("use_4bit", True)
    
    print(f"Loading {model_name} ({model_id})...")
    
    # Configure quantization
    load_kwargs = {
        "device_map": "auto",
        "trust_remote_code": True,
    }
    
    if use_4bit and torch.cuda.is_available():
        print("  Using 4-bit quantization...")
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
        )
        load_kwargs["quantization_config"] = bnb_config
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Load model
    model = AutoModelForCausalLM.from_pretrained(model_id, **load_kwargs)
    model.eval()
    
    # Cache
    _model_cache[model_name] = model
    _tokenizer_cache[model_name] = tokenizer
    
    print(f"  Loaded successfully!")
    return model, tokenizer

In [ ]:
def run_inference(
    prompt: str,
    model_name: str = "medgemma_4b",
    max_new_tokens: int = 512,
    temperature: float = 0.2,
    do_sample: bool = True,
) -> str:
    """
    Run inference on a loaded model.
    
    Args:
        prompt: Input prompt
        model_name: Which model to use
        max_new_tokens: Maximum tokens to generate
        temperature: Sampling temperature
        do_sample: Whether to use sampling
    
    Returns:
        Generated text (completion only, not including prompt)
    """
    model, tokenizer = load_model(model_name)
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature if do_sample else None,
            do_sample=do_sample,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # Decode only the generated part
    generated_ids = outputs[0, inputs["input_ids"].shape[1]:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True)
    
    return response.strip()

## 4. Load Primary Model (MedGemma 4B)

In [ ]:
# Load the primary model
print("Loading MedGemma 4B IT (primary model for all agents)...")
model, tokenizer = load_model("medgemma_4b")

# Quick test
test_response = run_inference(
    "What is ESBL? Answer in one sentence.",
    model_name="medgemma_4b",
    max_new_tokens=100,
)
print(f"\nTest response: {test_response}")

## 5. Utility Functions

In [ ]:
# Creatinine Clearance Calculator (Cockcroft-Gault equation)

def calculate_crcl(
    age_years: float,
    weight_kg: float,
    serum_creatinine_mg_dl: float,
    sex: Literal["male", "female"],
    height_cm: Optional[float] = None,
) -> float:
    """
    Calculate Creatinine Clearance using the Cockcroft-Gault equation.
    
    Formula: CrCl = [(140 - age) x weight x (0.85 if female)] / (72 x SCr)
    """
    if serum_creatinine_mg_dl <= 0:
        raise ValueError("Serum creatinine must be positive")
    
    crcl = ((140 - age_years) * weight_kg) / (72 * serum_creatinine_mg_dl)
    
    if sex == "female":
        crcl *= 0.85
    
    return round(crcl, 1)


def get_renal_dose_category(crcl: float) -> str:
    """Categorize renal function for dosing."""
    if crcl >= 90:
        return "normal"
    elif crcl >= 60:
        return "mild_impairment"
    elif crcl >= 30:
        return "moderate_impairment"
    elif crcl >= 15:
        return "severe_impairment"
    else:
        return "esrd"


# Test CrCl calculation
test_crcl = calculate_crcl(age_years=65, weight_kg=70, serum_creatinine_mg_dl=1.2, sex="male")
print(f"Test CrCl: {test_crcl} mL/min ({get_renal_dose_category(test_crcl)})")

In [ ]:
import re

def safe_json_parse(text: str) -> Optional[Dict[str, Any]]:
    """Safely parse JSON from agent output, handling markdown code blocks."""
    if not text:
        return None
    
    # Try direct parse
    try:
        return json.loads(text)
    except json.JSONDecodeError:
        pass
    
    # Try extracting from markdown code blocks
    patterns = [
        r"```json\s*\n?(.*?)\n?```",
        r"```\s*\n?(.*?)\n?```",
        r"\{[\s\S]*\}",
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text, re.DOTALL)
        if match:
            try:
                json_str = match.group(1) if match.lastindex else match.group(0)
                return json.loads(json_str)
            except (json.JSONDecodeError, IndexError):
                continue
    
    return None

## 6. Agent Prompt Templates

In [ ]:
# Agent 1: Intake Historian
INTAKE_HISTORIAN_SYSTEM = """You are an expert clinical intake specialist. Your role is to:

1. Parse and structure patient demographics and clinical history
2. Calculate Creatinine Clearance (CrCl) using the Cockcroft-Gault equation when data is available
3. Identify key risk factors for antimicrobial-resistant infections
4. Determine the appropriate treatment stage (empirical vs targeted)

RISK FACTORS TO IDENTIFY:
- Prior MRSA or MDR infection history
- Recent antibiotic use (within 90 days)
- Healthcare-associated vs community-acquired infection
- Immunocompromised status
- Recent hospitalization or ICU stay
- Presence of medical devices (catheters, lines)
- Renal or hepatic impairment

OUTPUT FORMAT:
Provide a structured JSON response with the following fields:
{
    "patient_summary": "Brief clinical summary",
    "creatinine_clearance_ml_min": <number or null>,
    "renal_dose_adjustment_needed": <boolean>,
    "identified_risk_factors": ["list", "of", "factors"],
    "suspected_pathogens": ["list", "of", "likely", "organisms"],
    "infection_severity": "mild|moderate|severe|critical",
    "recommended_stage": "empirical|targeted",
    "notes": "Any additional clinical observations"
}
"""

INTAKE_HISTORIAN_PROMPT = """Analyze the following patient information and provide a structured clinical assessment.

PATIENT DATA:
{patient_data}

CURRENT MEDICATIONS:
{medications}

KNOWN ALLERGIES:
{allergies}

CLINICAL CONTEXT:
- Suspected infection site: {infection_site}
- Suspected source: {suspected_source}

Provide your structured assessment following the system instructions."""

In [ ]:
# Agent 2: Vision Specialist
VISION_SPECIALIST_SYSTEM = """You are an expert medical laboratory data extraction specialist. Your role is to:

1. Extract structured data from laboratory reports (culture & sensitivity, antibiograms)
2. Handle reports in ANY language - always output in English
3. Identify pathogens, antibiotics tested, MIC values, and S/I/R interpretations
4. Flag any critical or unusual findings

OUTPUT FORMAT:
Provide a structured JSON response:
{
    "specimen_type": "blood|urine|wound|respiratory|other",
    "collection_date": "YYYY-MM-DD or null",
    "identified_organisms": [
        {
            "organism_name": "Standardized English name",
            "colony_count": "if available",
            "significance": "pathogen|colonizer|contaminant"
        }
    ],
    "susceptibility_results": [
        {
            "organism": "Organism name",
            "antibiotic": "Standardized antibiotic name",
            "mic_value": <number or null>,
            "mic_unit": "mg/L",
            "interpretation": "S|I|R"
        }
    ],
    "critical_findings": ["List of urgent findings"],
    "extraction_confidence": 0.0-1.0
}
"""

VISION_SPECIALIST_PROMPT = """Extract structured laboratory data from the following report.

REPORT CONTENT:
{report_content}

Extract all pathogen identifications, susceptibility results, and MIC values.
Always standardize to English medical terminology.
Flag any critical findings that require urgent attention.

Provide your structured extraction following the system instructions."""

In [ ]:
# Agent 3: Trend Analyst
TREND_ANALYST_SYSTEM = """You are an expert antimicrobial resistance trend analyst. Your role is to:

1. Analyze MIC trends over time to detect "MIC Creep"
2. Calculate resistance velocity and predict treatment failure risk
3. Compare current MICs against EUCAST/CLSI breakpoints
4. Identify emerging resistance patterns

RISK STRATIFICATION:
- LOW: Stable MIC, well below breakpoint (>4x margin)
- MODERATE: Rising trend but still 2-4x below breakpoint
- HIGH: Approaching breakpoint (<2x margin) or rapid increase
- CRITICAL: At or above breakpoint, or >4-fold increase over baseline

OUTPUT FORMAT:
{
    "organism": "Pathogen name",
    "antibiotic": "Antibiotic name",
    "baseline_mic": <number>,
    "current_mic": <number>,
    "fold_change": <number>,
    "trend": "stable|increasing|decreasing",
    "breakpoint_susceptible": <number>,
    "margin_to_breakpoint": <number>,
    "risk_level": "LOW|MODERATE|HIGH|CRITICAL",
    "recommendation": "Continue current therapy|Consider alternatives|Urgent switch needed",
    "rationale": "Detailed explanation"
}
"""

TREND_ANALYST_PROMPT = """Analyze the MIC trend data and assess resistance risk.

ORGANISM: {organism}
ANTIBIOTIC: {antibiotic}

HISTORICAL MIC DATA:
{mic_history}

EUCAST BREAKPOINT (S <=): {breakpoint} mg/L

Analyze the trend, calculate risk level, and provide recommendations.
Follow the system instructions for output format."""

In [ ]:
# Agent 4: Clinical Pharmacologist
CLINICAL_PHARMACOLOGIST_SYSTEM = """You are an expert clinical pharmacologist specializing in infectious diseases and antimicrobial stewardship. Your role is to:

1. Synthesize all available clinical data into a final antibiotic recommendation
2. Apply WHO AWaRe classification principles (ACCESS -> WATCH -> RESERVE)
3. Perform comprehensive drug safety checks
4. Adjust dosing for renal function

PRESCRIBING PRINCIPLES:
1. Start narrow, escalate only when justified
2. De-escalate when culture results allow
3. Prefer ACCESS category antibiotics when appropriate
4. Consider pharmacokinetic/pharmacodynamic (PK/PD) optimization

OUTPUT FORMAT:
{
    "primary_recommendation": {
        "antibiotic": "Drug name",
        "dose": "Amount and unit",
        "route": "IV|PO|IM",
        "frequency": "Dosing interval",
        "duration": "Treatment duration",
        "aware_category": "ACCESS|WATCH|RESERVE"
    },
    "alternative_recommendation": {
        "antibiotic": "Alternative drug",
        "indication": "When to use alternative"
    },
    "dose_adjustments": {
        "renal": "Adjustment details or None needed"
    },
    "safety_alerts": [
        {
            "level": "INFO|WARNING|CRITICAL",
            "message": "Alert message"
        }
    ],
    "monitoring_parameters": ["Labs/vitals to monitor"],
    "rationale": "Clinical reasoning",
    "guideline_references": ["Supporting guidelines"]
}
"""

CLINICAL_PHARMACOLOGIST_PROMPT = """Synthesize all clinical data and provide a final antibiotic recommendation.

PATIENT SUMMARY:
{intake_summary}

LAB RESULTS:
{lab_results}

MIC TREND ANALYSIS:
{trend_analysis}

PATIENT PARAMETERS:
- Age: {age} years
- Weight: {weight} kg
- CrCl: {crcl} mL/min
- Allergies: {allergies}
- Current medications: {current_medications}

INFECTION CONTEXT:
- Site: {infection_site}
- Severity: {severity}

Provide your final recommendation following the system instructions."""

## 7. Agent Implementation

In [ ]:
# State type definition
from typing import TypedDict, NotRequired

class InfectionState(TypedDict, total=False):
    """Global state for the Med-I-C pipeline."""
    # Patient data
    patient_id: Optional[str]
    age_years: Optional[float]
    sex: Optional[Literal["male", "female"]]
    weight_kg: Optional[float]
    height_cm: Optional[float]
    
    # Clinical context
    suspected_source: Optional[str]
    comorbidities: List[str]
    medications: List[str]
    allergies: List[str]
    infection_site: Optional[str]
    
    # Renal function
    serum_creatinine_mg_dl: Optional[float]
    creatinine_clearance_ml_min: Optional[float]
    
    # Lab data
    labs_raw_text: Optional[str]
    mic_data: List[Dict[str, Any]]
    
    # Stage routing
    stage: Literal["empirical", "targeted"]
    route_to_vision: bool
    route_to_trend_analyst: bool
    
    # Agent outputs
    intake_notes: Optional[str]
    vision_notes: Optional[str]
    trend_notes: Optional[str]
    pharmacology_notes: Optional[str]
    recommendation: Optional[Dict[str, Any]]
    
    # Safety
    safety_warnings: List[str]
    errors: List[str]

In [ ]:
def run_intake_historian(state: InfectionState) -> InfectionState:
    """
    Agent 1: Parse patient data, calculate CrCl, identify risk factors.
    """
    print("\n" + "="*60)
    print("AGENT 1: INTAKE HISTORIAN")
    print("="*60)
    
    # Calculate CrCl if we have required data
    crcl = None
    if all([state.get("age_years"), state.get("weight_kg"), 
            state.get("serum_creatinine_mg_dl"), state.get("sex")]):
        crcl = calculate_crcl(
            age_years=state["age_years"],
            weight_kg=state["weight_kg"],
            serum_creatinine_mg_dl=state["serum_creatinine_mg_dl"],
            sex=state["sex"],
        )
        state["creatinine_clearance_ml_min"] = crcl
        print(f"Calculated CrCl: {crcl} mL/min ({get_renal_dose_category(crcl)})")
    
    # Build patient data string
    patient_data = f"""
Age: {state.get('age_years', 'Unknown')} years
Sex: {state.get('sex', 'Unknown')}
Weight: {state.get('weight_kg', 'Unknown')} kg
Serum Creatinine: {state.get('serum_creatinine_mg_dl', 'Unknown')} mg/dL
CrCl: {crcl or 'Not calculated'} mL/min
Comorbidities: {', '.join(state.get('comorbidities', [])) or 'None reported'}
"""
    
    # Build prompt
    prompt = f"{INTAKE_HISTORIAN_SYSTEM}\n\n{INTAKE_HISTORIAN_PROMPT.format(
        patient_data=patient_data,
        medications=', '.join(state.get('medications', [])) or 'None reported',
        allergies=', '.join(state.get('allergies', [])) or 'No known allergies',
        infection_site=state.get('infection_site', 'Unknown'),
        suspected_source=state.get('suspected_source', 'Unknown'),
    )}"""
    
    # Run inference
    print("Running MedGemma inference...")
    response = run_inference(prompt, model_name="medgemma_4b", max_new_tokens=1024)
    
    # Parse response
    parsed = safe_json_parse(response)
    if parsed:
        state["intake_notes"] = json.dumps(parsed, indent=2)
        state["stage"] = parsed.get("recommended_stage", "empirical")
        print(f"\nIntake Assessment:")
        print(json.dumps(parsed, indent=2))
    else:
        state["intake_notes"] = response
        state["stage"] = "empirical"
        print(f"\nRaw response: {response[:500]}...")
    
    # Determine routing
    state["route_to_vision"] = bool(state.get("labs_raw_text"))
    print(f"\nStage: {state['stage']}")
    print(f"Route to Vision Specialist: {state['route_to_vision']}")
    
    return state

In [ ]:
def run_vision_specialist(state: InfectionState) -> InfectionState:
    """
    Agent 2: Extract structured data from lab reports.
    """
    print("\n" + "="*60)
    print("AGENT 2: VISION SPECIALIST")
    print("="*60)
    
    labs_raw = state.get("labs_raw_text", "")
    if not labs_raw:
        print("No lab data to process, skipping.")
        state["vision_notes"] = "No lab data provided"
        state["route_to_trend_analyst"] = False
        return state
    
    # Build prompt
    prompt = f"{VISION_SPECIALIST_SYSTEM}\n\n{VISION_SPECIALIST_PROMPT.format(
        report_content=labs_raw,
    )}"
    
    # Run inference
    print("Running MedGemma inference on lab report...")
    response = run_inference(prompt, model_name="medgemma_4b", max_new_tokens=2048)
    
    # Parse response
    parsed = safe_json_parse(response)
    if parsed:
        state["vision_notes"] = json.dumps(parsed, indent=2)
        
        # Extract MIC data
        susceptibility = parsed.get("susceptibility_results", [])
        state["mic_data"] = susceptibility
        state["route_to_trend_analyst"] = len(susceptibility) > 0
        
        print(f"\nExtracted Lab Data:")
        print(json.dumps(parsed, indent=2))
        
        # Check for critical findings
        critical = parsed.get("critical_findings", [])
        if critical:
            print(f"\n⚠️ CRITICAL FINDINGS: {critical}")
            state.setdefault("safety_warnings", []).extend(critical)
    else:
        state["vision_notes"] = response
        state["route_to_trend_analyst"] = False
        print(f"\nRaw response: {response[:500]}...")
    
    print(f"\nMIC data points: {len(state.get('mic_data', []))}")
    print(f"Route to Trend Analyst: {state.get('route_to_trend_analyst', False)}")
    
    return state

In [ ]:
def run_trend_analyst(state: InfectionState) -> InfectionState:
    """
    Agent 3: Analyze MIC trends and detect resistance velocity.
    """
    print("\n" + "="*60)
    print("AGENT 3: TREND ANALYST")
    print("="*60)
    
    mic_data = state.get("mic_data", [])
    if not mic_data:
        print("No MIC data to analyze, skipping.")
        state["trend_notes"] = "No MIC data available"
        return state
    
    trend_results = []
    
    for mic in mic_data:
        organism = mic.get("organism", "Unknown")
        antibiotic = mic.get("antibiotic", "Unknown")
        mic_value = mic.get("mic_value")
        
        if mic_value is None:
            continue
        
        # For demo, create synthetic history showing increasing trend
        mic_history = json.dumps([
            {"date": "2024-01-01", "mic_value": float(mic_value) / 2},
            {"date": "2024-06-01", "mic_value": float(mic_value) / 1.5},
            {"date": "2025-01-01", "mic_value": float(mic_value)},
        ], indent=2)
        
        # Use standard EUCAST breakpoint (simplified)
        breakpoint = 2.0  # Default S <= 2 mg/L
        
        prompt = f"{TREND_ANALYST_SYSTEM}\n\n{TREND_ANALYST_PROMPT.format(
            organism=organism,
            antibiotic=antibiotic,
            mic_history=mic_history,
            breakpoint=breakpoint,
        )}"
        
        print(f"\nAnalyzing: {organism} / {antibiotic}...")
        response = run_inference(prompt, model_name="medgemma_4b", max_new_tokens=1024)
        
        parsed = safe_json_parse(response)
        if parsed:
            trend_results.append(parsed)
            risk_level = parsed.get("risk_level", "UNKNOWN")
            print(f"  Risk Level: {risk_level}")
            
            if risk_level in ["HIGH", "CRITICAL"]:
                warning = f"MIC trend alert for {organism}/{antibiotic}: {parsed.get('recommendation', 'Review needed')}"
                state.setdefault("safety_warnings", []).append(warning)
        else:
            trend_results.append({"organism": organism, "antibiotic": antibiotic, "raw": response[:200]})
    
    state["trend_notes"] = json.dumps(trend_results, indent=2)
    print(f"\nTrend Analysis Complete:")
    print(json.dumps(trend_results, indent=2))
    
    return state

In [ ]:
def run_clinical_pharmacologist(state: InfectionState) -> InfectionState:
    """
    Agent 4: Generate final antibiotic recommendation with safety checks.
    """
    print("\n" + "="*60)
    print("AGENT 4: CLINICAL PHARMACOLOGIST")
    print("="*60)
    
    # Gather previous agent outputs
    intake_summary = state.get("intake_notes", "No intake data")
    lab_results = state.get("vision_notes", "No lab data")
    trend_analysis = state.get("trend_notes", "No trend data")
    
    prompt = f"{CLINICAL_PHARMACOLOGIST_SYSTEM}\n\n{CLINICAL_PHARMACOLOGIST_PROMPT.format(
        intake_summary=intake_summary,
        lab_results=lab_results,
        trend_analysis=trend_analysis,
        age=state.get('age_years', 'Unknown'),
        weight=state.get('weight_kg', 'Unknown'),
        crcl=state.get('creatinine_clearance_ml_min', 'Unknown'),
        allergies=', '.join(state.get('allergies', [])) or 'No known allergies',
        current_medications=', '.join(state.get('medications', [])) or 'None',
        infection_site=state.get('infection_site', 'Unknown'),
        severity='moderate',
    )}"
    
    print("Running MedGemma inference for final recommendation...")
    response = run_inference(prompt, model_name="medgemma_4b", max_new_tokens=2048)
    
    parsed = safe_json_parse(response)
    if parsed:
        state["pharmacology_notes"] = json.dumps(parsed, indent=2)
        
        # Build recommendation
        primary = parsed.get("primary_recommendation", {})
        recommendation = {
            "primary_antibiotic": primary.get("antibiotic"),
            "dose": primary.get("dose"),
            "route": primary.get("route"),
            "frequency": primary.get("frequency"),
            "duration": primary.get("duration"),
            "rationale": parsed.get("rationale"),
            "references": parsed.get("guideline_references", []),
            "safety_alerts": [a.get("message") for a in parsed.get("safety_alerts", [])],
        }
        
        alt = parsed.get("alternative_recommendation", {})
        if alt.get("antibiotic"):
            recommendation["backup_antibiotic"] = alt.get("antibiotic")
        
        state["recommendation"] = recommendation
        
        print(f"\n" + "="*60)
        print("FINAL RECOMMENDATION")
        print("="*60)
        print(json.dumps(recommendation, indent=2))
        
        # Add safety alerts
        for alert in parsed.get("safety_alerts", []):
            if alert.get("level") in ["WARNING", "CRITICAL"]:
                state.setdefault("safety_warnings", []).append(alert.get("message"))
    else:
        state["pharmacology_notes"] = response
        state["recommendation"] = {"rationale": response}
        print(f"\nRaw response: {response[:500]}...")
    
    return state

## 8. Pipeline Orchestration with LangGraph

In [ ]:
from langgraph.graph import StateGraph, END

def build_infection_graph():
    """
    Build the LangGraph StateGraph for the infection lifecycle workflow.
    
    Stage 1 (Empirical): Intake Historian -> Clinical Pharmacologist
    Stage 2 (Targeted): Intake Historian -> Vision Specialist -> Trend Analyst -> Clinical Pharmacologist
    """
    graph = StateGraph(InfectionState)
    
    # Add nodes
    graph.add_node("intake_historian", run_intake_historian)
    graph.add_node("vision_specialist", run_vision_specialist)
    graph.add_node("trend_analyst", run_trend_analyst)
    graph.add_node("clinical_pharmacologist", run_clinical_pharmacologist)
    
    # Set entry point
    graph.set_entry_point("intake_historian")
    
    # Conditional routing after intake
    def route_after_intake(state: InfectionState):
        if state.get("stage") == "targeted" and state.get("route_to_vision"):
            return "vision_specialist"
        return "clinical_pharmacologist"
    
    graph.add_conditional_edges(
        "intake_historian",
        route_after_intake,
        {
            "vision_specialist": "vision_specialist",
            "clinical_pharmacologist": "clinical_pharmacologist",
        }
    )
    
    # Conditional routing after vision
    def route_after_vision(state: InfectionState):
        if state.get("route_to_trend_analyst"):
            return "trend_analyst"
        return "clinical_pharmacologist"
    
    graph.add_conditional_edges(
        "vision_specialist",
        route_after_vision,
        {
            "trend_analyst": "trend_analyst",
            "clinical_pharmacologist": "clinical_pharmacologist",
        }
    )
    
    # Edges to final node
    graph.add_edge("trend_analyst", "clinical_pharmacologist")
    graph.add_edge("clinical_pharmacologist", END)
    
    return graph


def run_pipeline(patient_data: dict, labs_raw_text: Optional[str] = None) -> InfectionState:
    """
    Run the full infection lifecycle pipeline.
    
    Args:
        patient_data: Patient information dict
        labs_raw_text: Optional lab report text (triggers Stage 2)
    
    Returns:
        Final InfectionState with recommendation
    """
    # Build initial state
    initial_state: InfectionState = {
        "age_years": patient_data.get("age_years"),
        "weight_kg": patient_data.get("weight_kg"),
        "height_cm": patient_data.get("height_cm"),
        "sex": patient_data.get("sex"),
        "serum_creatinine_mg_dl": patient_data.get("serum_creatinine_mg_dl"),
        "medications": patient_data.get("medications", []),
        "allergies": patient_data.get("allergies", []),
        "comorbidities": patient_data.get("comorbidities", []),
        "infection_site": patient_data.get("infection_site"),
        "suspected_source": patient_data.get("suspected_source"),
        "safety_warnings": [],
        "errors": [],
    }
    
    # Add lab data if provided
    if labs_raw_text:
        initial_state["labs_raw_text"] = labs_raw_text
        initial_state["stage"] = "targeted"
    else:
        initial_state["stage"] = "empirical"
    
    # Build and run graph
    print("\n" + "#"*70)
    print(f"# STARTING MED-I-C PIPELINE (Stage: {initial_state['stage'].upper()})")
    print("#"*70)
    
    graph = build_infection_graph()
    compiled = graph.compile()
    final_state = compiled.invoke(initial_state)
    
    print("\n" + "#"*70)
    print("# PIPELINE COMPLETE")
    print("#"*70)
    
    return final_state

## 9. Test Cases

### Test Case 1: Stage 1 (Empirical) - Community UTI

In [ ]:
# Test Case 1: Stage 1 Empirical - Community UTI
patient_data_uti = {
    "age_years": 65,
    "weight_kg": 70,
    "height_cm": 170,
    "sex": "male",
    "serum_creatinine_mg_dl": 1.2,
    "medications": ["metformin", "lisinopril", "aspirin"],
    "allergies": ["penicillin"],
    "comorbidities": ["diabetes", "hypertension"],
    "infection_site": "urinary",
    "suspected_source": "community-acquired UTI",
}

result_uti = run_pipeline(patient_data_uti)

In [ ]:
# Display results
print("\n" + "="*70)
print("TEST CASE 1: COMMUNITY UTI (EMPIRICAL)")
print("="*70)

print(f"\nCrCl: {result_uti.get('creatinine_clearance_ml_min')} mL/min")
print(f"Stage: {result_uti.get('stage')}")

rec = result_uti.get('recommendation', {})
if rec:
    print(f"\nRecommendation:")
    print(f"  Drug: {rec.get('primary_antibiotic')}")
    print(f"  Dose: {rec.get('dose')}")
    print(f"  Route: {rec.get('route')}")
    print(f"  Frequency: {rec.get('frequency')}")
    print(f"  Duration: {rec.get('duration')}")
    print(f"  Rationale: {rec.get('rationale')}")

warnings = result_uti.get('safety_warnings', [])
if warnings:
    print(f"\nSafety Warnings:")
    for w in warnings:
        print(f"  ⚠️ {w}")

### Test Case 2: Stage 2 (Targeted) - With Lab Results

In [ ]:
# Test Case 2: Stage 2 Targeted - UTI with Lab Results
patient_data_targeted = {
    "age_years": 72,
    "weight_kg": 65,
    "height_cm": 165,
    "sex": "female",
    "serum_creatinine_mg_dl": 1.5,
    "medications": ["warfarin", "amlodipine"],
    "allergies": [],
    "comorbidities": ["atrial fibrillation", "hypertension", "CKD stage 3"],
    "infection_site": "urinary",
    "suspected_source": "complicated UTI with pyelonephritis",
}

lab_report = """
URINE CULTURE REPORT
Patient ID: 12345
Collection Date: 2025-02-15

Specimen: Midstream urine
Colony Count: >100,000 CFU/mL

ORGANISM ISOLATED:
Escherichia coli

ANTIMICROBIAL SUSCEPTIBILITY:
-----------------------------------
Antibiotic          MIC (mg/L)   Interpretation
-----------------------------------
Ampicillin          >32          R
Amoxicillin-Clav    16           I
Ceftriaxone         0.25         S
Cefepime            0.5          S
Ciprofloxacin       0.5          S
Levofloxacin        1            S
Nitrofurantoin      32           S
TMP-SMX             >4           R
Gentamicin          2            S
Meropenem           0.06         S

NOTES:
- ESBL screening negative
- No carbapenemase detected
"""

result_targeted = run_pipeline(patient_data_targeted, labs_raw_text=lab_report)

In [ ]:
# Display results
print("\n" + "="*70)
print("TEST CASE 2: COMPLICATED UTI WITH LAB RESULTS (TARGETED)")
print("="*70)

print(f"\nCrCl: {result_targeted.get('creatinine_clearance_ml_min')} mL/min")
print(f"Stage: {result_targeted.get('stage')}")

print(f"\nExtracted MIC Data:")
for mic in result_targeted.get('mic_data', []):
    print(f"  - {mic.get('organism')} / {mic.get('antibiotic')}: MIC {mic.get('mic_value')} ({mic.get('interpretation')})")

rec = result_targeted.get('recommendation', {})
if rec:
    print(f"\nFinal Recommendation:")
    print(f"  Primary: {rec.get('primary_antibiotic')}")
    print(f"  Dose: {rec.get('dose')}")
    print(f"  Route: {rec.get('route')}")
    print(f"  Frequency: {rec.get('frequency')}")
    print(f"  Duration: {rec.get('duration')}")
    if rec.get('backup_antibiotic'):
        print(f"  Alternative: {rec.get('backup_antibiotic')}")
    print(f"  Rationale: {rec.get('rationale')}")

warnings = result_targeted.get('safety_warnings', [])
if warnings:
    print(f"\n⚠️ Safety Warnings:")
    for w in warnings:
        print(f"  - {w}")

### Test Case 3: ESBL-producing Organism (High-Risk)

In [ ]:
# Test Case 3: ESBL E. coli
patient_esbl = {
    "age_years": 58,
    "weight_kg": 85,
    "height_cm": 175,
    "sex": "male",
    "serum_creatinine_mg_dl": 1.0,
    "medications": ["metformin", "atorvastatin"],
    "allergies": [],
    "comorbidities": ["diabetes", "recent hospitalization"],
    "infection_site": "bloodstream",
    "suspected_source": "healthcare-associated bacteremia",
}

lab_esbl = """
BLOOD CULTURE REPORT
Collection Date: 2025-02-18

POSITIVE: Gram-negative bacilli

FINAL IDENTIFICATION:
Escherichia coli (ESBL-producing)

ANTIMICROBIAL SUSCEPTIBILITY:
-----------------------------------
Antibiotic          MIC (mg/L)   Interpretation
-----------------------------------
Ampicillin          >32          R
Ampicillin-Sulbact  >32          R
Ceftriaxone         >32          R
Cefepime            >32          R
Ceftazidime         >32          R
Ciprofloxacin       >4           R
Levofloxacin        >8           R
TMP-SMX             >4           R
Gentamicin          8            I
Amikacin            4            S
Ertapenem           0.25         S
Meropenem           0.06         S
Imipenem            0.5          S
Tigecycline         0.5          S

ESBL CONFIRMATION: POSITIVE
Carbapenemase: NOT DETECTED

CRITICAL ALERT: ESBL-producing organism in bloodstream
"""

result_esbl = run_pipeline(patient_esbl, labs_raw_text=lab_esbl)

In [ ]:
# Display ESBL results
print("\n" + "="*70)
print("TEST CASE 3: ESBL E. coli BACTEREMIA (HIGH-RISK)")
print("="*70)

rec = result_esbl.get('recommendation', {})
if rec:
    print(f"\nRecommendation:")
    print(f"  Primary: {rec.get('primary_antibiotic')}")
    print(f"  Dose: {rec.get('dose')}")
    print(f"  Route: {rec.get('route')}")
    print(f"  Rationale: {rec.get('rationale')}")

warnings = result_esbl.get('safety_warnings', [])
if warnings:
    print(f"\n🚨 SAFETY ALERTS:")
    for w in warnings:
        print(f"  - {w}")

## 10. Streamlit App (Optional - for local testing)

Note: Streamlit doesn't run directly in Kaggle notebooks. To test the Streamlit app:
1. Download this notebook and the source files
2. Run locally with `streamlit run app.py`

Alternatively, you can use `ngrok` or `localtunnel` to expose the Streamlit app.

In [ ]:
# Optional: Install and run Streamlit with localtunnel
# Uncomment to use

# !pip install -q streamlit localtunnel

# # Write the app file
# app_code = '''
# import streamlit as st
# import json

# st.set_page_config(page_title="Med-I-C: AMR-Guard", page_icon="🦠", layout="wide")

# st.title("🦠 Med-I-C: AMR-Guard")
# st.subheader("Infection Lifecycle Orchestrator - Multi-Agent System")

# st.markdown("""
# This demo showcases the Med-I-C multi-agent system powered by MedGemma.

# **Note:** Running in demo mode. For full functionality, deploy with GPU support.
# """)

# # Patient form
# with st.form("patient_form"):
#     col1, col2 = st.columns(2)
#     with col1:
#         age = st.number_input("Age", 0, 120, 65)
#         weight = st.number_input("Weight (kg)", 1.0, 300.0, 70.0)
#         sex = st.selectbox("Sex", ["male", "female"])
#     with col2:
#         creatinine = st.number_input("Creatinine (mg/dL)", 0.1, 20.0, 1.2)
#         infection_site = st.selectbox("Infection Site", ["urinary", "respiratory", "bloodstream"])
#     
#     submitted = st.form_submit_button("Get Recommendation")

# if submitted:
#     st.success("Demo mode: Showing simulated recommendation")
#     st.json({
#         "primary_antibiotic": "Ciprofloxacin",
#         "dose": "500mg",
#         "route": "PO",
#         "frequency": "Every 12 hours",
#         "duration": "7 days"
#     })
# '''

# with open('streamlit_app.py', 'w') as f:
#     f.write(app_code)

# # Run with localtunnel
# !streamlit run streamlit_app.py &>/dev/null &
# !npx localtunnel --port 8501

## 11. Summary & Conclusions

This notebook demonstrates the **Med-I-C** multi-agent system for antimicrobial stewardship:

### Key Features:
1. **4 Specialized Agents** powered by MedGemma 4B IT
2. **Conditional Routing** via LangGraph for Stage 1 (Empirical) vs Stage 2 (Targeted)
3. **CrCl Calculation** using Cockcroft-Gault equation
4. **MIC Trend Analysis** for resistance detection
5. **Safety Checks** including drug interactions and allergy alerts

### Models Used:
- **MedGemma 4B IT** - Primary model for all agents (4-bit quantized)
- **TxGemma 2B** - Optional safety validation (not demonstrated in this notebook)

### Future Enhancements:
- Integration with RAG (ChromaDB) for guideline retrieval
- MedGemma 27B for complex trend analysis
- Vision capabilities for image-based lab report extraction
- Regional resistance pattern analysis

---

**Competition:** MedGemma Impact Challenge  
**Category:** Agentic Workflow  
**Deadline:** February 24, 2026

In [ ]:
# Final memory cleanup
import gc
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU Memory after cleanup: {torch.cuda.memory_allocated() / 1e9:.2f} GB")